# Récupération et traitement des paroles des cantiques H&C

Notebook épuré: une seule cellule de code exécute le scraping complet.
- Détection de chaque cantique via son `h2` contenant `Cantique X`
- Chaque couplet détecté via `p.Clustermoyen` (titre: Cantique X — verset/strophe Y)
- Lignes du couplet: toutes les `p` avec classe contenant `posie`/`poesie` jusqu'au prochain titre
- Sortie: un fichier JSON par cantique + `all_cantiques.json` suivant le format de `exemple.json`.


In [18]:
# Cellule unique de scraping - exécuter pour générer les JSON
# Sorties écrites dans data/extracted/: cantique_XXX.json + all_cantiques.json
import re, json, unicodedata
from pathlib import Path
import requests
from bs4 import BeautifulSoup, NavigableString

URL = "https://www.cantiquest.org/HeC-Paroles/HeC-001a271_Paroles.htm"
OUTPUT_DIR = Path(r"d:/Analyse_cantiques/data/extracted")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

resp = requests.get(URL, timeout=60, verify=False)
resp.raise_for_status()
soup = BeautifulSoup(resp.content, 'html.parser')

H2_NUM_RE = re.compile(r"Cantique\s+(\d+)", re.IGNORECASE)
COUPLET_RE = re.compile(r"^Cantique\s+(\d+)\s+[—\-–]?\s*(?:verset|strophe)\s+(\d+)\s*$", re.IGNORECASE)

chants = {}
current_num = None
current_couplet_lines = []

def flush_couplet():
    global current_couplet_lines
    if current_num is None:
        current_couplet_lines = []
        return
    if current_couplet_lines:
        texte = '\n'.join(line for line in current_couplet_lines if line.strip())
        if texte.strip():
            chants[current_num]['couplets'].append({'texte': texte})
    current_couplet_lines = []

for h2 in soup.find_all('h2'):
    raw_h2 = h2.get_text(' ', strip=True)
    m_num = H2_NUM_RE.search(raw_h2)
    if not m_num:
        continue
    flush_couplet()
    current_num = int(m_num.group(1))
    chants.setdefault(current_num, {
        'numero': current_num,
        'auteur': None,
        'couplets': []
    })

    node = h2.next_sibling
    while node:
        if isinstance(node, NavigableString):
            node = node.next_sibling
            continue
        if getattr(node, 'name', None) == 'h2':
            break
        if node.name == 'p':
            classes = node.get('class') or []
            text = unicodedata.normalize('NFKC', node.get_text(' ', strip=True).replace('\xa0', ' ')).strip()
            if 'Clustermoyen' in classes:
                if COUPLET_RE.match(text):
                    flush_couplet()
                node = node.next_sibling
                continue
            if any('posie' in c.lower() or 'poesie' in c.lower() for c in classes):
                if text:
                    current_couplet_lines.append(text)
        node = node.next_sibling

flush_couplet()

for c in chants.values():
    c['nbre_couplets'] = len(c['couplets'])

print('Nombre total de cantiques:', len(chants))
print('Cantique 1 - nombre de couplets:', chants.get(1, {}).get('nbre_couplets'))

for num, data in chants.items():
    with open(OUTPUT_DIR / f"cantique_{num:03d}.json", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
with open(OUTPUT_DIR / 'all_cantiques.json', 'w', encoding='utf-8') as f:
    json.dump(sorted(chants.values(), key=lambda x: x['numero']), f, ensure_ascii=False, indent=2)

chants.get(1)

d:\Analyse_cantiques\hc\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cantiquest.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Nombre total de cantiques: 271
Cantique 1 - nombre de couplets: 2


{'numero': 1,
 'auteur': None,
 'couplets': [{'texte': 'Nous t’adorons, notre Père,\nÔ toi qui vis dans la lumière,\nEt d’un regard sondes les cieux.\nRéunis en ta présence,\nObjets de ton amour immense,\nNous louons ton nom glorieux.\nRefrain :\nÀ toi louange, honneur,\nTout-puissant Créateur,\nAlléluia !\nOui, gloire, honneur,\nAu Dieu Sauveur !\nAlléluia ! Alléluia !'},
  {'texte': 'Sur nous a brillé ta face,\nDieu Souverain, Père de grâce,\nEn Jésus, Fils de ton amour.\nPar Lui notre âme bénie,\nT’adore, ô Dieu, te glorifie,\nEt te célèbre en ce séjour,\n{ À toi louange, honneur,\nTout-puissant Créateur, } etc'}],
 'nbre_couplets': 2}